In [ ]:
# Task 4: η effects and metrics (2 features)
# Run this cell/file independently

# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Utilities
def plot_histories(histories, title, ylabel):
    plt.figure(figsize=(7,5))
    for label, values in histories.items():
        plt.plot(range(1, len(values)+1), values, label=str(label))
    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

def make_iris_binary(two_features=True):
    iris = load_iris()
    X = iris.data
    y = iris.target
    mask = (y == 0) | (y == 1)
    X = X[mask]
    y = y[mask]
    y = np.where(y == 1, 1, -1)
    if two_features:
        X = X[:, [2, 3]]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
    )
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

# Models
class AdalineManual:
    def __init__(self, lr=0.01, epochs=100, random_state=0):
        self.lr = lr
        self.epochs = epochs
        self.random_state = random_state
        self.w_ = None
        self.b_ = 0.0
        self.sse_history_ = []

    def fit(self, X, y):
        rng = np.random.default_rng(self.random_state)
        self.w_ = rng.normal(0.0, 0.01, size=X.shape[1])
        self.b_ = 0.0
        self.sse_history_.clear()
        for _ in range(self.epochs):
            net = X @ self.w_ + self.b_
            errors = y - net
            self.w_ += self.lr * (X.T @ errors) / X.shape[0]
            self.b_ += self.lr * errors.mean()
            self.sse_history_.append(float(np.sum(errors**2)))
        return self

    def predict(self, X):
        return np.where(X @ self.w_ + self.b_ >= 0.0, 1, -1)

class PerceptronManual:
    def __init__(self, lr=1.0, epochs=30, random_state=0):
        self.lr = lr
        self.epochs = epochs
        self.random_state = random_state
        self.w_ = None
        self.b_ = 0.0
        self.mistakes_ = []

    def fit(self, X, y):
        rng = np.random.default_rng(self.random_state)
        self.w_ = rng.normal(0.0, 0.01, size=X.shape[1])
        self.b_ = 0.0
        self.mistakes_.clear()
        for _ in range(self.epochs):
            idx = rng.permutation(X.shape[0])
            mistakes = 0
            for i in idx:
                xi, yi = X[i], y[i]
                if yi * (xi @ self.w_ + self.b_) <= 0:
                    self.w_ += self.lr * yi * xi
                    self.b_ += self.lr * yi
                    mistakes += 1
            self.mistakes_.append(mistakes)
        return self

    def predict(self, X):
        return np.where(X @ self.w_ + self.b_ >= 0.0, 1, -1)

# Run
if __name__ == "__main__":
    Xtr, Xte, ytr, yte = make_iris_binary(two_features=True)
    etas = [0.001, 0.01, 0.1, 1.0]

    hist_adaline, rows_adaline = {}, []
    for eta in etas:
        m = AdalineManual(lr=eta, epochs=100, random_state=RANDOM_STATE).fit(Xtr, ytr)
        hist_adaline[f"η={eta}"] = m.sse_history_
        pred = m.predict(Xte)
        acc = accuracy_score(yte, pred)
        prec, rec, f1, _ = precision_recall_fscore_support(
            yte, pred, average="binary", pos_label=1, zero_division=0
        )
        rows_adaline.append({"Model":"ADALINE", "η":eta, "Accuracy":acc, "Precision":prec, "Recall":rec, "F1":f1})

    plot_histories(hist_adaline, "ADALINE — SSE vs Epochs (η sweep)", ylabel="SSE")
    df_adaline = pd.DataFrame(rows_adaline).sort_values(["F1","Accuracy"], ascending=[False, False])
    print("\n=== η Effects — ADALINE (2 features) ===")
    print(df_adaline.to_string(index=False))

    hist_perc, rows_perc = {}, []
    for eta in etas:
        p = PerceptronManual(lr=eta, epochs=30, random_state=RANDOM_STATE).fit(Xtr, ytr)
        hist_perc[f"η={eta}"] = p.mistakes_
        pred = p.predict(Xte)
        acc = accuracy_score(yte, pred)
        prec, rec, f1, _ = precision_recall_fscore_support(
            yte, pred, average="binary", pos_label=1, zero_division=0
        )
        rows_perc.append({"Model":"Perceptron", "η":eta, "Accuracy":acc, "Precision":prec, "Recall":rec, "F1":f1})

    plot_histories(hist_perc, "Perceptron — Mistakes per Epoch (η sweep)", ylabel="Mistakes/epoch")
    df_perc = pd.DataFrame(rows_perc).sort_values(["F1","Accuracy"], ascending=[False, False])
    print("\n=== η Effects — Perceptron (2 features) ===")
    print(df_perc.to_string(index=False))

    best_a = df_adaline.iloc[0].to_dict()
    best_p = df_perc.iloc[0].to_dict()
    print("\n=== Best η Summary ===")
    print(pd.DataFrame([best_a, best_p]).to_string(index=False))
